In [31]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the data from the CSV file
running_data = pd.read_csv('data/running_activities_header.csv')

running_data['activity_date'] = pd.to_datetime(running_data['start_date_local'])

# Display the loaded data
running_data.head()


,id,name,distance,moving_time,elapsed_time,total_elevation_gain,type,start_date,start_date_local,timezone,...,weighted_average_watts,kilojoules,device_watts,has_heartrate,average_heartrate,max_heartrate,pr_count,total_photo_count,has_kudoed,activity_date
0,11419814645,AM Lift + PM Run,9670.0,2780.0,2968.0,164.0,Run,2024-05-15 20:21:30+00:00,2024-05-15 16:21:30,America/New_York,...,335.0,931.8,True,True,148.1,163.0,0,0,False,2024-05-15 16:21:30
1,11407789781,Tempo,10016.4,2715.0,2846.0,107.0,Run,2024-05-14 11:19:30+00:00,2024-05-14 07:19:30,America/New_York,...,348.0,942.1,True,True,150.4,169.0,0,0,False,2024-05-14 07:19:30
2,11403956766,Recovery,4846.2,1944.0,1960.0,84.0,Run,2024-05-13 21:21:33+00:00,2024-05-13 17:21:33,America/New_York,...,262.0,488.0,True,True,107.5,157.0,0,0,False,2024-05-13 17:21:33
3,11393434570,Morning Run,15981.9,4963.0,5104.0,201.0,Run,2024-05-12 11:39:05+00:00,2024-05-12 07:39:05,America/New_York,...,318.0,1556.6,True,True,133.3,162.0,0,0,False,2024-05-12 07:39:05
4,11385534272,Morning Run,10142.8,2710.0,2713.0,27.0,Run,2024-05-11 13:16:03+00:00,2024-05-11 09:16:03,America/New_York,...,351.0,944.7,True,True,151.6,169.0,3,0,False,2024-05-11 09:16:03


In [34]:
marathon_dates = {'NYE': '2022-12-31',
                'Pittsburgh': '2023-05-07',
                'Detroit': '2023-10-15',
                'Clearwater': '2024-01-21',
                'Big Sur': '2024-04-28'}
training_blocks = {}
# calculate 16 weeks prior to each marathon
for marathon, date in marathon_dates.items():
    print(f'{marathon} is on {date}')
    training_blocks[marathon] = pd.to_datetime(date) - pd.DateOffset(weeks=16)
    marathon_dates[marathon] = pd.to_datetime(date)
    print(f'Training block starts on {training_blocks[marathon]}\n')

NYE is on 2022-12-31
Training block starts on 2022-09-10 00:00:00

Pittsburgh is on 2023-05-07
Training block starts on 2023-01-15 00:00:00

Detroit is on 2023-10-15
Training block starts on 2023-06-25 00:00:00

Clearwater is on 2024-01-21
Training block starts on 2023-10-01 00:00:00

Big Sur is on 2024-04-28
Training block starts on 2024-01-07 00:00:00



In [39]:
training_start_date = training_blocks['NYE'] - pd.Timedelta(weeks=18)

# Filter the running activities within the training period
training_data = running_data[
    (running_data['activity_date'] >= training_start_date) &
    (running_data['activity_date'] <= marathon_dates['NYE'])
]

# Creating a week start column.
training_data['week_start'] = training_data['activity_date'] - pd.to_timedelta(training_data['activity_date'].dt.weekday, unit='d')

# Calculate weekly metrics: total mileage, average pace, and average heart rate
weekly_stats = training_data.resample('W-MON', on='week_start').agg({
    'distance': 'sum',
    'moving_time': 'sum',
    'elapsed_time': 'sum',
    'average_heartrate': 'mean'
}).reset_index()

# Convert the distance from meters to miles
weekly_stats['distance_mi'] = weekly_stats['distance'] / 1609.34

# Calculate the average pace in minutes per mile
weekly_stats['pace_min_per_mi'] = (weekly_stats['elapsed_time'] / 60) / weekly_stats['distance_mi']

weekly_stats[['week_start', 'distance_mi', 'pace_min_per_mi', 'average_heartrate']]



C:\Users\mfalc\AppData\Local\Temp\ipykernel_28120\503547518.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data['week_start'] = training_data['activity_date'] - pd.to_timedelta(training_data['activity_date'].dt.weekday, unit='d')


,week_start,distance_mi,pace_min_per_mi,average_heartrate
0,2022-05-02,2.009209,9.655542,NaN
1,2022-05-09,4.601141,11.243588,NaN
2,2022-05-16,11.227460,9.068540,NaN
3,2022-05-23,11.967328,9.524543,NaN
4,2022-05-30,12.461879,9.073003,NaN
5,2022-06-06,10.238607,8.467949,NaN
6,2022-06-13,2.004238,9.080759,NaN
7,2022-06-20,6.007307,10.609301,NaN
8,2022-06-27,0.000000,NaN,NaN
9,2022-07-04,12.653572,11.194467,NaN
